**06_EDAoffeatures**


used this container: 
ml:1.0

In [1]:
# load packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sns
import math

from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator,TransformerMixin

from pandas_profiling import ProfileReport

from statistics import mean
import math
from scipy import stats

%matplotlib inline

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
if __name__ == "__main__":
    
    #this is for the outlier trimming
    
    class OutlierRemover(BaseEstimator,TransformerMixin):
        def __init__(self,factor=1.5):
            self.factor = factor

        def outlier_detector(self,X,y=None):
            X = pd.Series(X).copy()
            q1 = X.quantile(0.1)
            q3 = X.quantile(0.9)
            iqr = q3 - q1
            self.lower_bound.append(q1 - (self.factor * iqr))
            self.upper_bound.append(q3 + (self.factor * iqr))

        def fit(self,X,y=None):
            self.lower_bound = []
            self.upper_bound = []
            X.apply(self.outlier_detector)
            return self

        def transform(self,X,y=None):
            X = pd.DataFrame(X).copy()
            for i in range(X.shape[1]):
                x = X.iloc[:, i].copy()
                x.loc[x < self.lower_bound[i]] = self.lower_bound[i]
                x.loc[x > self.upper_bound[i]] = self.upper_bound[i]
                X.iloc[:, i] = x
            return X

    outlier_remover = OutlierRemover()

In [3]:
base_dir = '/home/jovyan/work/Goldfinch/ML_paper/'
suffix = 'raw_data/intermediate_files/'

df = pd.read_excel(f'{base_dir}raw_data/DKD_Abstracted_Data_v1.0_2021.04.27.xlsx', engine='openpyxl')
df_map = pd.read_csv(f'{base_dir}raw_data/Tuttle_source_GRE_id_mapping.csv')
df = pd.merge(df, df_map, left_on = "CURE-CKD Patient ID", right_on = "source_subject_id", how = 'inner')

# read in dataframe of dataset with all patients included and all features

In [4]:
# read input file
input_df_path = f'{base_dir}raw_data/preprocessed_data/ef_agg_before_biopsy_0_years_0_gap.parquet'
dfraw = pd.read_parquet(input_df_path, engine='pyarrow')

# read the proceedure dictionary
proc_df = pd.read_csv(f'{base_dir}raw_data/proc_mapping_file.csv')
proc_dic = proc_df.set_index('procedure_code')['short_description'].to_dict()
proc_dic.update({"791.0": "Proteinuria", "250.00": "Type 2 diabetes mellitus without complications", "99211": "Office/outpatient visit-99211" , "99212": "Office/outpatient visit-99212", 
            "99213": "Office/outpatient visit-99213", "99214": "Office/outpatient visit-99214", "99215": "Office/outpatient visit-99215"})

#read the colnames
colnames = pd.read_csv(f'{base_dir}{suffix}0_years_0/column_header.csv', header = None)
colnames = colnames[0].tolist()

# read the df to patient selection
#dftest = pd.read_csv(f'{base_dir}{suffix}ML_target_variables_test.csv' , index=False, header=True)
dftest = pd.read_csv(f'{base_dir}{suffix}ML_target_variables_test.csv' )
dfvaltrain = pd.read_csv(f'{base_dir}{suffix}ML_target_variables_trainval.csv' )

dfpat = pd.concat([dfvaltrain, dftest], ignore_index=True)
dfpat = dfpat[['gfb_subject_id', 'is_DN', 'duration_dm', 'Diabetic_Retinopathy', 'age_biopsy']]

#select test subject ids
df = pd.merge(dfraw, dfpat, left_on = 'gfb_subject_id', right_on = 'gfb_subject_id', how = 'inner')

#select relevant columns
clean_colnames = list(set(colnames) - set(df.columns.tolist()))
clean_colnames = pd.Series(clean_colnames).str.replace('_TRUE|_>10 years|_>5 years|_Not Reported|_Yes|_White/Caucasian|_Male|_Not Hispanic or Latino',\
                                '', regex=True)
clean_colnames = list(set(clean_colnames)) #this gets rid of duplicates
clean_colnames = list(clean_colnames) + list(set(colnames) & set(df.columns.tolist()))

df_clean = df[clean_colnames]

# add proceedure names
colnames = df_clean.columns.tolist()
colnames = [proc_dic.get(item,item)  for item in colnames]
df_clean.columns = colnames

# this is to remove/windsor the outliers
df_cat = df_clean.select_dtypes(include= 'object')
df_num = df_clean.select_dtypes(include='float')
filt_num_feat = df_num.columns.tolist()

outl = outlier_remover.fit_transform(df_num)
df_outl = pd.DataFrame(outl, columns = filt_num_feat)

df_clean_nooutl = pd.concat([df_outl, df_cat], axis=1)

In [5]:
# raw version
profile = ProfileReport(df_clean, title='ML Features Profiling report', explorative=True, correlations={"cramers": {"calculate": False}})
profile.to_file(f'{base_dir}{suffix}ML_Features_Profiling_report.html')

# no outliers
profile = ProfileReport(df_clean_nooutl, title='ML Features Profiling report no outliers', explorative=True, correlations={"cramers": {"calculate": False}})
profile.to_file(f'{base_dir}{suffix}ML_Features_Profiling_report_no_outl.html')

/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)
<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if 

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# save as .csv

df_clean.to_csv(f"{base_dir}{suffix}clean_dataset.csv", header=True, index=False)
df_clean_nooutl.to_csv(f"{base_dir}{suffix}clean_dataset_no_outlier.csv", header=True, index=False)